In [7]:
import numpy as np
import pandas as pd
import scipy as sc
import os
from datetime import date
from os import listdir
from os.path import isfile, join
import math
from scipy.optimize import curve_fit

In [ ]:
path_ = 'C:\\Users\\mehrd\\PycharmProjects\\drinkometer\\data\\2021\\dataframes\\26'
df_m = pd.read_excel("{}\drinkometer_df_26.xlsx".format(path_))

gender = ['female']*14 + ['male']*16
animals = np.concatenate([np.arange(1001,1007), np.arange(1008,1016), np.arange(1017,1033)])
state = [str('dark'), str('light')]

In [ ]:
df1_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1001_daily_{}.xlsx".format(path_, gender[0], state[0]))
df2_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1002_daily_{}.xlsx".format(path_, gender[1],state[0]))
df3_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1003_daily_{}.xlsx".format(path_, gender[2],state[0]))
df4_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1004_daily_{}.xlsx".format(path_, gender[3],state[0]))
df5_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1005_daily_{}.xlsx".format(path_, gender[4],state[0]))
df6_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1006_daily_{}.xlsx".format(path_, gender[5],state[0]))
df8_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1008_daily_{}.xlsx".format(path_, gender[6],state[0]))
df9_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1009_daily_{}.xlsx".format(path_, gender[7],state[0]))
df10_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1010_daily_{}.xlsx".format(path_, gender[8],state[0]))
df11_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1011_daily_{}.xlsx".format(path_, gender[9],state[0]))
df12_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1012_daily_{}.xlsx".format(path_, gender[10],state[0]))
df13_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1013_daily_{}.xlsx".format(path_, gender[11],state[0]))
df14_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1014_daily_{}.xlsx".format(path_, gender[12],state[0]))
#df15_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1015_daily_{}.xlsx".format(path_, gender[13],state[0]))]
df17_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1017_daily_{}.xlsx".format(path_, gender[14],state[0]))
df18_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1018_daily_{}.xlsx".format(path_, gender[15],state[0]))
df19_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1019_daily_{}.xlsx".format(path_, gender[16],state[0]))
df20_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1020_daily_{}.xlsx".format(path_, gender[17],state[0]))
df21_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1021_daily_{}.xlsx".format(path_, gender[18],state[0]))
df22_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1022_daily_{}.xlsx".format(path_, gender[19],state[0]))
df23_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1023_daily_{}.xlsx".format(path_, gender[20],state[0]))
df24_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1024_daily_{}.xlsx".format(path_, gender[21],state[0]))
df25_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1025_daily_{}.xlsx".format(path_, gender[22],state[0]))
df26_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1026_daily_{}.xlsx".format(path_, gender[23],state[0]))
df27_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1027_daily_{}.xlsx".format(path_, gender[24],state[0]))
df28_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1028_daily_{}.xlsx".format(path_, gender[25],state[0]))
df29_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1029_daily_{}.xlsx".format(path_, gender[26],state[0]))
df30_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1030_daily_{}.xlsx".format(path_, gender[27],state[0]))
df31_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1031_daily_{}.xlsx".format(path_, gender[28],state[0]))
df32_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1032_daily_{}.xlsx".format(path_, gender[29],state[0]))


df1_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1001_daily_{}.xlsx".format(path_, gender[0], state[1]))
df2_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1002_daily_{}.xlsx".format(path_, gender[1],state[1]))
df3_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1003_daily_{}.xlsx".format(path_, gender[2],state[1]))
df4_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1004_daily_{}.xlsx".format(path_, gender[3],state[1]))
df5_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1005_daily_{}.xlsx".format(path_, gender[4],state[1]))
df6_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1006_daily_{}.xlsx".format(path_, gender[5],state[1]))
df8_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1008_daily_{}.xlsx".format(path_, gender[6],state[1]))
df9_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1009_daily_{}.xlsx".format(path_, gender[7],state[1]))
df10_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1010_daily_{}.xlsx".format(path_, gender[8],state[1]))
df11_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1011_daily_{}.xlsx".format(path_, gender[9],state[1]))
df12_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1012_daily_{}.xlsx".format(path_, gender[10],state[1]))
df13_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1013_daily_{}.xlsx".format(path_, gender[11],state[1]))
df14_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1014_daily_{}.xlsx".format(path_, gender[12],state[1]))
#df15_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1015_daily_{}.xlsx".format(path_, gender[13],state[1]))]
df17_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1017_daily_{}.xlsx".format(path_, gender[14],state[1]))
df18_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1018_daily_{}.xlsx".format(path_, gender[15],state[1]))
df19_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1019_daily_{}.xlsx".format(path_, gender[16],state[1]))
df20_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1020_daily_{}.xlsx".format(path_, gender[17],state[1]))
df21_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1021_daily_{}.xlsx".format(path_, gender[18],state[1]))
df22_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1022_daily_{}.xlsx".format(path_, gender[19],state[1]))
df23_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1023_daily_{}.xlsx".format(path_, gender[20],state[1]))
df24_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1024_daily_{}.xlsx".format(path_, gender[21],state[1]))
df25_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1025_daily_{}.xlsx".format(path_, gender[22],state[1]))
df26_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1026_daily_{}.xlsx".format(path_, gender[23],state[1]))
df27_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1027_daily_{}.xlsx".format(path_, gender[24],state[1]))
df28_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1028_daily_{}.xlsx".format(path_, gender[25],state[1]))
df29_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1029_daily_{}.xlsx".format(path_, gender[26],state[1]))
df30_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1030_daily_{}.xlsx".format(path_, gender[27],state[1]))
df31_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1031_daily_{}.xlsx".format(path_, gender[28],state[1]))
df32_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1032_daily_{}.xlsx".format(path_, gender[29],state[1]))


In [ ]:
df_m = pd.DataFrame(index=range(2*7), columns=['date', 'day_index', 'strain', 'state', 'oxytocin', 'quinine',
                                                'w_1', 'a5_1', 'a10_1', 'a20_1', 'loco_1',
                                                'w_2', 'a5_2', 'a10_2', 'a20_2', 'loco_2',
                                                'w_3', 'a5_3', 'a10_3', 'a20_3', 'loco_3',
                                                'w_4', 'a5_4', 'a10_4', 'a20_4', 'loco_4',
                                                'w_5', 'a5_5', 'a10_5', 'a20_5', 'loco_5',
                                                'w_6', 'a5_6', 'a10_6', 'a20_6', 'loco_6',
                                                'w_8', 'a5_8', 'a10_8', 'a20_8', 'loco_8',
                                                'w_9', 'a5_9', 'a10_9', 'a20_9', 'loco_9',
                                                'w_10', 'a5_10', 'a10_10', 'a20_10', 'loco_10',
                                                'w_11', 'a5_11', 'a10_11', 'a20_11', 'loco_11',
                                                'w_12', 'a5_12', 'a10_12', 'a20_12', 'loco_12',
                                                'w_13', 'a5_13', 'a10_13', 'a20_13', 'loco_13',
                                                'w_14', 'a5_14', 'a10_14', 'a20_14', 'loco_14',
                                                'w_15', 'a5_15', 'a10_15', 'a20_15', 'loco_15',
                                                'w_17', 'a5_17', 'a10_17', 'a20_17', 'loco_17',
                                                'w_18', 'a5_18', 'a10_18', 'a20_18', 'loco_18',
                                                'w_19', 'a5_19', 'a10_19', 'a20_19', 'loco_19',
                                                'w_20', 'a5_20', 'a10_20', 'a20_20', 'loco_20',
                                                'w_21', 'a5_21', 'a10_21', 'a20_21', 'loco_21',
                                                'w_22', 'a5_22', 'a10_22', 'a20_22', 'loco_22',
                                                'w_23', 'a5_23', 'a10_23', 'a20_23', 'loco_23',
                                                'w_24', 'a5_24', 'a10_24', 'a20_24', 'loco_24',
                                                'w_25', 'a5_25', 'a10_25', 'a20_25', 'loco_25',
                                                'w_26', 'a5_26', 'a10_26', 'a20_26', 'loco_26',
                                                'w_27', 'a5_27', 'a10_27', 'a20_27', 'loco_27',
                                                'w_28', 'a5_28', 'a10_28', 'a20_28', 'loco_28',
                                                'w_29', 'a5_29', 'a10_29', 'a20_29', 'loco_29',
                                                'w_30', 'a5_30', 'a10_30', 'a20_30', 'loco_30',
                                                'w_31', 'a5_31', 'a10_31', 'a20_31', 'loco_31',
                                                'w_32', 'a5_32', 'a10_32', 'a20_32', 'loco_32'])


for i in range(7):
    df_m.loc[2*i, 'state'] = str('dark')
    df_m.loc[2*i+1, 'state'] = str('light')
    df_m.loc[2*i, 'date'] = df1_l.loc[i, 'date']
    df_m.loc[2*i+1, 'date'] = df1_l.loc[i, 'date']
    df_m.loc[2*i, 'day_index'] = df1_l.loc[i, 'day_index']
    df_m.loc[2*i+1, 'day_index'] = df1_l.loc[i, 'day_index']
    df_m.loc[2*i, 'strain'] = df1_l.loc[i, 'strain']
    df_m.loc[2*i+1, 'strain'] = df1_l.loc[i, 'strain']
    df_m.loc[2*i, 'oxytocin'] = df1_l.loc[i, 'oxytocin']
    df_m.loc[2*i+1, 'oxytocin'] = df1_l.loc[i, 'oxytocin']
    df_m.loc[2*i, 'quinine'] = df1_l.loc[i, 'quinine']
    df_m.loc[2*i+1, 'quinine'] = df1_l.loc[i, 'quinine']
    
    df_m.loc[2*i, 'w_1'] = df1_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_1'] = df1_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_1'] = df1_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_1'] = df1_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_1'] = df1_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_1'] = df1_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_1'] = df1_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_1'] = df1_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_1'] = df1_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_1'] = df1_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_2'] = df2_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_2'] = df2_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_2'] = df2_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_2'] = df2_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_2'] = df2_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_2'] = df2_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_2'] = df2_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_2'] = df2_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_2'] = df2_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_2'] = df2_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_3'] = df3_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_3'] = df3_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_3'] = df3_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_3'] = df3_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_3'] = df3_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_3'] = df3_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_3'] = df3_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_3'] = df3_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_3'] = df3_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_3'] = df3_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_4'] = df4_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_4'] = df4_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_4'] = df4_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_4'] = df4_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_4'] = df4_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_4'] = df4_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_4'] = df4_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_4'] = df4_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_4'] = df4_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_4'] = df4_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_5'] = df5_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_5'] = df5_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_5'] = df5_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_5'] = df5_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_5'] = df5_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_5'] = df5_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_5'] = df5_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_5'] = df5_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_5'] = df5_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_5'] = df5_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_6'] = df6_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_6'] = df6_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_6'] = df6_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_6'] = df6_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_6'] = df6_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_6'] = df6_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_6'] = df6_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_6'] = df6_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_6'] = df6_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_6'] = df6_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_8'] = df8_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_8'] = df8_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_8'] = df8_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_8'] = df8_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_8'] = df8_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_8'] = df8_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_8'] = df8_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_8'] = df8_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_8'] = df8_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_8'] = df8_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_9'] = df9_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_9'] = df9_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_9'] = df9_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_9'] = df9_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_9'] = df9_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_9'] = df9_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_9'] = df9_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_9'] = df9_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_9'] = df9_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_9'] = df9_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_10'] = df10_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_10'] = df10_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_10'] = df10_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_10'] = df10_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_10'] = df10_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_10'] = df10_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_10'] = df10_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_10'] = df10_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_10'] = df10_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_10'] = df10_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_11'] = df11_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_11'] = df11_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_11'] = df11_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_11'] = df11_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_11'] = df11_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_11'] = df11_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_11'] = df11_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_11'] = df11_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_11'] = df11_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_11'] = df11_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_12'] = df12_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_12'] = df12_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_12'] = df12_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_12'] = df12_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_12'] = df12_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_12'] = df12_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_12'] = df12_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_12'] = df12_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_12'] = df12_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_12'] = df12_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_13'] = df13_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_13'] = df13_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_13'] = df13_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_13'] = df13_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_13'] = df13_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_13'] = df13_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_13'] = df13_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_13'] = df13_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_13'] = df13_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_13'] = df13_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_14'] = df14_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_14'] = df14_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_14'] = df14_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_14'] = df14_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_14'] = df14_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_14'] = df14_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_14'] = df14_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_14'] = df14_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_14'] = df14_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_14'] = df14_l.loc[i, 'locomotive']
    
    #df_m.loc[2*i, 'w_15'] = df15_d.loc[i, 'water']
    #df_m.loc[2*i+1, 'w_15'] = df15_l.loc[i, 'water']
    #df_m.loc[2*i, 'a5_15'] = df15_d.loc[i, 'alcohol_5']
    #df_m.loc[2*i+1, 'a5_15'] = df15_l.loc[i, 'alcohol_5']
    #df_m.loc[2*i, 'a10_15'] = df15_d.loc[i, 'alcohol_10']
    #df_m.loc[2*i+1, 'a10_15'] = df15_l.loc[i, 'alcohol_10']
    #df_m.loc[2*i, 'a20_15'] = df15_d.loc[i, 'alcohol_20']
    #df_m.loc[2*i+1, 'a20_15'] = df15_l.loc[i, 'alcohol_20']
    #df_m.loc[2*i, 'loco_15'] = df15_d.loc[i, 'locomotive']
    #df_m.loc[2*i+1, 'loco_15'] = df15_l.loc[i, 'locomotive']

    df_m.loc[2*i, 'w_17'] = df17_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_17'] = df17_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_17'] = df17_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_17'] = df17_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_17'] = df17_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_17'] = df17_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_17'] = df17_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_17'] = df17_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_17'] = df17_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_17'] = df17_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_18'] = df18_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_18'] = df18_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_18'] = df18_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_18'] = df18_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_18'] = df18_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_18'] = df18_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_18'] = df18_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_18'] = df18_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_18'] = df18_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_18'] = df18_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_19'] = df19_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_19'] = df19_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_19'] = df19_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_19'] = df19_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_19'] = df19_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_19'] = df19_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_19'] = df19_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_19'] = df19_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_19'] = df19_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_19'] = df19_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_20'] = df20_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_20'] = df20_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_20'] = df20_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_20'] = df20_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_20'] = df20_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_20'] = df20_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_20'] = df20_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_20'] = df20_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_20'] = df20_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_20'] = df20_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_21'] = df21_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_21'] = df21_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_21'] = df21_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_21'] = df21_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_21'] = df21_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_21'] = df21_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_21'] = df21_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_21'] = df21_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_21'] = df21_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_21'] = df21_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_22'] = df22_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_22'] = df22_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_22'] = df22_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_22'] = df22_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_22'] = df22_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_22'] = df22_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_22'] = df22_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_22'] = df22_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_22'] = df22_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_22'] = df22_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_23'] = df23_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_23'] = df23_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_23'] = df23_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_23'] = df23_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_23'] = df23_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_23'] = df23_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_23'] = df23_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_23'] = df23_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_23'] = df23_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_23'] = df23_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_24'] = df24_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_24'] = df24_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_24'] = df24_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_24'] = df24_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_24'] = df24_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_24'] = df24_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_24'] = df24_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_24'] = df24_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_24'] = df24_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_24'] = df24_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_25'] = df25_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_25'] = df25_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_25'] = df25_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_25'] = df25_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_25'] = df25_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_25'] = df25_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_25'] = df25_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_25'] = df25_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_25'] = df25_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_25'] = df25_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_26'] = df26_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_26'] = df26_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_26'] = df26_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_26'] = df26_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_26'] = df26_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_26'] = df26_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_26'] = df26_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_26'] = df26_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_26'] = df26_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_26'] = df26_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_27'] = df27_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_27'] = df27_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_27'] = df27_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_27'] = df27_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_27'] = df27_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_27'] = df27_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_27'] = df27_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_27'] = df27_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_27'] = df27_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_27'] = df27_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_28'] = df28_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_28'] = df28_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_28'] = df28_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_28'] = df28_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_28'] = df28_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_28'] = df28_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_28'] = df28_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_28'] = df28_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_28'] = df28_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_28'] = df28_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_29'] = df29_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_29'] = df29_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_29'] = df29_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_29'] = df29_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_29'] = df29_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_29'] = df29_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_29'] = df29_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_29'] = df29_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_29'] = df29_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_29'] = df29_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_30'] = df30_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_30'] = df30_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_30'] = df30_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_30'] = df30_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_30'] = df30_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_30'] = df30_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_30'] = df30_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_30'] = df30_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_30'] = df30_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_30'] = df30_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_31'] = df31_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_31'] = df31_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_31'] = df31_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_31'] = df31_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_31'] = df31_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_31'] = df31_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_31'] = df31_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_31'] = df31_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_31'] = df31_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_31'] = df31_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_32'] = df32_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_32'] = df32_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_32'] = df32_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_32'] = df32_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_32'] = df32_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_32'] = df32_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_32'] = df32_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_32'] = df32_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_32'] = df32_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_32'] = df32_l.loc[i, 'locomotive']

In [ ]:
df_m.to_excel(r'{}\drinkometer_df_26.xlsx'.format(path_), index=False, header=True)

In [ ]:
path_ = 'C:\\Users\\mehrd\\PycharmProjects\\drinkometer\\data\\2021\\dataframes\\25'
df_25 = pd.read_excel("{}\drinkometer_df_25.xlsx".format(path_))

path_ = 'C:\\Users\\mehrd\\PycharmProjects\\drinkometer\\data\\2021\\dataframes\\26'
df_26 = pd.read_excel("{}\drinkometer_df_26.xlsx".format(path_))

In [ ]:
df_a_state = pd.DataFrame(index=range(7*30*2), columns=['day_index', 'strain', 'gender',
                                                'oxytocin', 'quinine',
                                                'condition', 'alcohol'])

days = ([1]*30 + [2]*30 + [3]*30 + [4]*30 + [5]*30 + [6]*30 + [7]*30)*2
gender = ((['female']*14 + ['male']*16)*7)*2
strain = [str('wistar')] * 7 * 30 * 2
oxy = ['before'] * 210 + ['active'] * 210
condition = (['inject'] + ['inject'] + ['nasal'] + ['inject'] + ['nasal'] +
            ['control'] + ['nasal'] + ['inject'] + ['control'] + ['nasal'] + 
            ['nasal'] + ['inject'] + ['control'] + ['control'] + ['control'] +
            ['nasal'] + ['nasal'] + ['nasal'] + ['control'] + ['inject'] +
            ['inject'] + ['inject'] + ['inject'] + ['inject'] + ['nasal'] +
            ['control'] + ['inject'] + ['control'] + ['nasal'] + ['control']) * 14

df_a_state['day_index'] = days
df_a_state['gender'] = gender
df_a_state['strain'] = strain
df_a_state['oxytocin'] = oxy
df_a_state['condition'] = condition

In [ ]:
counter = 0
for i in range(7):
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_1']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_1']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_1'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_2']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_2']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_2']) 
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_3']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_3']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_3'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_4']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_4']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_4'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_5']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_5']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_5'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_6']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_6']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_6'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_8']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_8']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_8'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_9']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_9']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_9'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_10']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_10']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_10'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_11']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_11']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_11'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_12']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_12']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_12'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_13']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_13']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_13'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_14']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_14']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_14'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_15']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_15']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_15'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_17']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_17']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_17'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_18']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_18']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_18'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_19']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_19']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_19'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_20']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_20']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_20'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_21']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_21']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_21'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_22']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_22']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_22'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_23']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_23']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_23'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_24']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_24']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_24'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_25']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_25']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_25'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_25']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_25']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_25'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_27']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_27']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_27'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_28']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_28']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_28'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_29']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_29']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_29'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_30']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_30']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_30'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_31']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_31']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_31'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_32']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_32']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_32'])
    counter += 1
    

for i in range(7):
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_1']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_1']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_1'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_2']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_2']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_2']) 
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_3']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_3']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_3'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_4']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_4']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_4'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_5']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_5']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_5'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_6']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_6']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_6'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_8']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_8']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_8'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_9']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_9']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_9'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_10']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_10']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_10'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_11']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_11']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_11'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_12']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_12']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_12'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_13']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_13']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_13'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_14']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_14']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_14'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_15']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_15']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_15'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_17']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_17']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_17'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_18']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_18']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_18'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_19']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_19']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_19'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_20']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_20']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_20'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_21']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_21']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_21'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_22']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_22']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_22'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_23']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_23']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_23'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_24']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_24']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_24'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_25']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_25']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_25'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_25']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_25']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_25'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_27']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_27']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_27'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_28']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_28']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_28'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_29']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_29']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_29'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_30']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_30']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_30'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_31']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_31']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_31'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_32']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_32']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_32'])
    counter += 1
        
        

In [ ]:
df_a_state.replace(0, np.nan, inplace=True)

In [ ]:
df_m = df_a_state.loc[(df_a_state['gender'] == str("male"))]
df_f = df_a_state.loc[(df_a_state['gender'] == str("female"))]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from statannot import add_stat_annotation

x = "condition"
y = "alcohol"
hue = "oxytocin"


ax = sns.boxplot(data=df_f, x=x, y=y, hue=hue)
add_stat_annotation(ax, data=df_f, x=x, y=y, hue=hue,
                    box_pairs=[(("inject", "before"), ("inject", "active")),
                                 (("nasal", "before"), ("nasal", "active")),
                                 (("control", "before"), ("control", "active"))
                                ],
                    test='t-test_ind', text_format='full', loc='inside', verbose=2)
plt.legend(loc='upper left', bbox_to_anchor=(1.03, 1))
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from statannot import add_stat_annotation

x = "condition"
y = "alcohol"
hue = "oxytocin"


ax = sns.boxplot(data=df_m, x=x, y=y, hue=hue)
add_stat_annotation(ax, data=df_m, x=x, y=y, hue=hue,
                    box_pairs=[(("inject", "before"), ("inject", "active")),
                                 (("nasal", "before"), ("nasal", "active")),
                                 (("control", "before"), ("control", "active"))
                                ],
                    test='t-test_ind', text_format='full', loc='inside', verbose=2)
plt.legend(loc='upper left', bbox_to_anchor=(1.03, 1))
plt.show()